In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data1 = pd.read_csv('data1.csv')
data2 = pd.read_csv('data2.csv')
data3 = pd.read_csv('data3.csv')
data4 = pd.read_csv('data4.csv')
data5 = pd.read_csv('data5.csv')
data6 = pd.read_csv('data6.csv')
data7 = pd.read_csv('data7.csv')
data8 = pd.read_csv('data8.csv')
data = pd.concat([data1,data2,data3,data4,data5,data6,data7,data8], ignore_index=True).dropna(axis=0)

In [3]:
data['IVS']=data['CVOL']-data['PVOL']
clean_return = [float(p.replace('%',''))/100 for p in data['return']]
data['return'] = clean_return
clean_future_return = [float(p.replace('%',''))/100 for p in data['future_return']]
data['future_return'] = clean_future_return

In [4]:
all_dates = data['date'].sort_values().unique()

## CALL

In [5]:
monthly_performance_call = pd.DataFrame({
    'date':all_dates
})
capm_alpha_call = pd.DataFrame({
    'date':all_dates
})
ff3_alpha_call = pd.DataFrame({
    'date':all_dates
})

In [6]:
for index,row in monthly_performance_call.iterrows():
    monthly_data = data[data['date']==row['date']].reset_index()
    monthly_data['decile'] = pd.qcut(monthly_data['delta_CVOL'],10,labels=False)
    for i in range(0,10):
        col_name = f"decile {i}"
        monthly_performance_call.at[index, col_name] = monthly_data[monthly_data['decile']==i]['future_return'].mean()
        capm_alpha_call.at[index, col_name] = monthly_data[monthly_data['decile']==i]['future_return'].mean()-monthly_data[monthly_data['decile']==i]['capm_ahpha'].mean()
        ff3_alpha_call.at[index, col_name] = monthly_data[monthly_data['decile']==i]['future_return'].mean()-monthly_data[monthly_data['decile']==i]['ff3_alpha'].mean()
monthly_performance_call['10-1'] = monthly_performance_call['decile 9']-monthly_performance_call['decile 0']
capm_alpha_call['10-1'] = capm_alpha_call['decile 9']-capm_alpha_call['decile 0']
ff3_alpha_call['10-1'] = ff3_alpha_call['decile 9']-ff3_alpha_call['decile 0']

In [7]:
monthly_return_call = []
capm_alpha_average_call = []
ff3_alpha_average_call = []
col_names = ['decile 0','decile 1','decile 2','decile 3','decile 4','decile 5','decile 6','decile 7','decile 8','decile 9','10-1']
for i in range(0,11):
    col_name = col_names[i]
    monthly_return_call.append(np.prod((monthly_performance_call[col_name])+1)**(1/58)-1)
    capm_alpha_average_call.append(capm_alpha_call[col_name].mean())
    ff3_alpha_average_call.append(ff3_alpha_call[col_name].mean())


In [8]:
t_return,p_return = stats.ttest_1samp(monthly_performance_call['10-1'],0)
t_capm,p_capm = stats.ttest_1samp(capm_alpha_call['10-1'],0)
t_ff3,p_ff3 = stats.ttest_1samp(ff3_alpha_call['10-1'],0)
monthly_return_call.append(t_return)
capm_alpha_average_call.append(t_capm)
ff3_alpha_average_call.append(t_ff3)

In [9]:
call_report = pd.DataFrame({
    'decile':['decile 0','decile 1','decile 2','decile 3','decile 4','decile 5','decile 6','decile 7','decile 8','decile 9','10-1','t-stat'],
    'return': monthly_return_call,
    'capm_alpha':capm_alpha_average_call,
    'ff3_alpha':ff3_alpha_average_call
})

In [10]:
call_report

,decile,return,capm_alpha,ff3_alpha
0,decile 0,0.002252,0.012530,0.012079
1,decile 1,0.000680,0.002934,0.001535
2,decile 2,0.003090,0.004598,0.003219
3,decile 3,0.005600,0.007330,0.006105
4,decile 4,0.004172,0.005656,0.004876
5,decile 5,0.005662,0.007492,0.006887
6,decile 6,0.004652,0.007220,0.005814
7,decile 7,0.003011,0.007159,0.005178
8,decile 8,0.003324,0.010072,0.007396
9,decile 9,0.004357,0.025280,0.022849


## PUT

In [11]:
monthly_performance_put = pd.DataFrame({
    'date':all_dates
})
capm_alpha_put = pd.DataFrame({
    'date':all_dates
})
ff3_alpha_put = pd.DataFrame({
    'date':all_dates
})

In [12]:
for index,row in monthly_performance_put.iterrows():
    monthly_data = data[data['date']==row['date']].reset_index()
    monthly_data['decile'] = pd.qcut(monthly_data['delta_PVOL'],10,labels=False)
    for i in range(0,10):
        col_name = f"decile {i}"
        monthly_performance_put.at[index, col_name] = monthly_data[monthly_data['decile']==i]['future_return'].mean()
        capm_alpha_put.at[index, col_name] = monthly_data[monthly_data['decile']==i]['future_return'].mean()-monthly_data[monthly_data['decile']==i]['capm_ahpha'].mean()
        ff3_alpha_put.at[index, col_name] = monthly_data[monthly_data['decile']==i]['future_return'].mean()-monthly_data[monthly_data['decile']==i]['ff3_alpha'].mean()
monthly_performance_put['10-1'] = monthly_performance_put['decile 9']-monthly_performance_put['decile 0']
capm_alpha_put['10-1'] = capm_alpha_put['decile 9']-capm_alpha_put['decile 0']
ff3_alpha_put['10-1'] = ff3_alpha_put['decile 9']-ff3_alpha_put['decile 0']

In [13]:
monthly_return_put = []
capm_alpha_average_put = []
ff3_alpha_average_put = []
col_names = ['decile 0','decile 1','decile 2','decile 3','decile 4','decile 5','decile 6','decile 7','decile 8','decile 9','10-1']
for i in range(0,11):
    col_name = col_names[i]
    monthly_return_put.append(np.prod((monthly_performance_put[col_name])+1)**(1/58)-1)
    capm_alpha_average_put.append(capm_alpha_put[col_name].mean())
    ff3_alpha_average_put.append(ff3_alpha_put[col_name].mean())


In [14]:
t_return,p_return = stats.ttest_1samp(monthly_performance_put['10-1'],0)
t_capm,p_capm = stats.ttest_1samp(capm_alpha_put['10-1'],0)
t_ff3,p_ff3 = stats.ttest_1samp(ff3_alpha_put['10-1'],0)
monthly_return_put.append(t_return)
capm_alpha_average_put.append(t_capm)
ff3_alpha_average_put.append(t_ff3)

In [15]:
put_report = pd.DataFrame({
    'decile':['decile 0','decile 1','decile 2','decile 3','decile 4','decile 5','decile 6','decile 7','decile 8','decile 9','10-1','t-stat'],
    'return': monthly_return_put,
    'capm_alpha':capm_alpha_average_put,
    'ff3_alpha':ff3_alpha_average_put
})

In [16]:
put_report

,decile,return,capm_alpha,ff3_alpha
0,decile 0,0.003392,0.013546,0.012841
1,decile 1,0.001625,0.003570,0.002368
2,decile 2,0.002931,0.005123,0.003863
3,decile 3,0.005305,0.006789,0.005545
4,decile 4,0.004893,0.007103,0.005971
5,decile 5,0.006103,0.008144,0.007324
6,decile 6,0.004524,0.006972,0.005674
7,decile 7,0.003456,0.007616,0.005719
8,decile 8,0.002412,0.008868,0.006606
9,decile 9,0.002130,0.022535,0.020023


## Correlation

In [17]:
data[['CVOL','PVOL','delta_CVOL','delta_PVOL','IVS']].corr(method='spearman')

,CVOL,PVOL,delta_CVOL,delta_PVOL,IVS
CVOL,1.000000,0.928133,0.193142,0.117391,0.287011
PVOL,0.928133,1.000000,0.112607,0.189406,0.043830
delta_CVOL,0.193142,0.112607,1.000000,0.555570,0.222808
delta_PVOL,0.117391,0.189406,0.555570,1.000000,-0.177691
IVS,0.287011,0.043830,0.222808,-0.177691,1.000000


## IVS

In [18]:
monthly_performance_IVS = pd.DataFrame({
    'date':all_dates
})
capm_alpha_IVS = pd.DataFrame({
    'date':all_dates
})
ff3_alpha_IVS = pd.DataFrame({
    'date':all_dates
})

In [19]:
for index,row in monthly_performance_IVS.iterrows():
    monthly_data = data[data['date']==row['date']].reset_index()
    monthly_data['decile'] = pd.qcut(monthly_data['IVS'],10,labels=False)
    for i in range(0,10):
        col_name = f"decile {i}"
        monthly_performance_IVS.at[index, col_name] = monthly_data[monthly_data['decile']==i]['future_return'].mean()
        capm_alpha_IVS.at[index, col_name] = monthly_data[monthly_data['decile']==i]['future_return'].mean()-monthly_data[monthly_data['decile']==i]['capm_ahpha'].mean()
        ff3_alpha_IVS.at[index, col_name] = monthly_data[monthly_data['decile']==i]['future_return'].mean()-monthly_data[monthly_data['decile']==i]['ff3_alpha'].mean()
monthly_performance_IVS['10-1'] = monthly_performance_IVS['decile 9']-monthly_performance_IVS['decile 0']
capm_alpha_IVS['10-1'] = capm_alpha_IVS['decile 9']-capm_alpha_IVS['decile 0']
ff3_alpha_IVS['10-1'] = ff3_alpha_IVS['decile 9']-ff3_alpha_IVS['decile 0']

In [20]:
monthly_return_IVS = []
capm_alpha_average_IVS = []
ff3_alpha_average_IVS = []
col_names = ['decile 0','decile 1','decile 2','decile 3','decile 4','decile 5','decile 6','decile 7','decile 8','decile 9','10-1']
for i in range(0,11):
    col_name = col_names[i]
    monthly_return_IVS.append(np.prod((monthly_performance_IVS[col_name])+1)**(1/58)-1)
    capm_alpha_average_IVS.append(capm_alpha_IVS[col_name].mean())
    ff3_alpha_average_IVS.append(ff3_alpha_IVS[col_name].mean())


In [21]:
t_return,p_return = stats.ttest_1samp(monthly_performance_IVS['10-1'],0)
t_capm,p_capm = stats.ttest_1samp(capm_alpha_IVS['10-1'],0)
t_ff3,p_ff3 = stats.ttest_1samp(ff3_alpha_IVS['10-1'],0)
monthly_return_IVS.append(t_return)
capm_alpha_average_IVS.append(t_capm)
ff3_alpha_average_IVS.append(t_ff3)

In [22]:
IVS_report = pd.DataFrame({
    'decile':['decile 0','decile 1','decile 2','decile 3','decile 4','decile 5','decile 6','decile 7','decile 8','decile 9','10-1','t-stat'],
    'return': monthly_return_IVS,
    'capm_alpha':capm_alpha_average_IVS,
    'ff3_alpha':ff3_alpha_average_IVS
})

In [23]:
IVS_report

,decile,return,capm_alpha,ff3_alpha
0,decile 0,-0.001478,0.014600,0.011779
1,decile 1,-0.000189,0.001324,0.000510
2,decile 2,0.004497,0.005222,0.004456
3,decile 3,0.006473,0.006030,0.005538
4,decile 4,0.004995,0.004819,0.004376
5,decile 5,0.003192,0.003215,0.002401
6,decile 6,0.003839,0.005525,0.004115
7,decile 7,0.004396,0.007600,0.005947
8,decile 8,0.006012,0.012527,0.010418
9,decile 9,0.005815,0.029387,0.026382


In [24]:
call_report.to_csv('CVOL_port.csv')
put_report.to_csv('PVOL_port.csv')
IVS_report.to_csv('IVS_port.csv')